In [32]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns

from sklearn import tree
from sklearn.model_selection import train_test_split

DIR = 'PATH/TO/YOUR/DATA'

In [33]:
X = pd.read_csv('application_train.csv')
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
X_ext = X[['SK_ID_CURR','EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'TARGET']]

In [35]:
for function_name in ['nanmin', 'nanmax', 'sum', 'mean', 'var', 'median', 'std', 'nanmedian', 'nanmean', 'min', 'max']:
            X_ext['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(
                X_ext[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: All-NaN axis encountered
  app.launch_new_instance()
C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: Mean of empty slice
  app.launch_new_instance()


In [36]:
X_ext.head()

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,external_sources_nanmin,external_sources_nanmax,external_sources_sum,external_sources_mean,external_sources_var,external_sources_median,external_sources_std,external_sources_nanmedian,external_sources_nanmean,external_sources_min,external_sources_max
0,100002,0.083037,0.262949,0.139376,1,0.083037,0.262949,0.485361,0.161787,0.005646,0.139376,0.075139,0.139376,0.161787,0.083037,0.262949
1,100003,0.311267,0.622246,NaN,0,0.311267,0.622246,0.933513,0.466757,0.024177,NaN,0.155489,0.466757,0.466757,0.311267,0.622246
2,100004,NaN,0.555912,0.729567,0,0.555912,0.729567,1.285479,0.642739,0.007539,NaN,0.086827,0.642739,0.642739,0.555912,0.729567
3,100006,NaN,0.650442,NaN,0,0.650442,0.650442,0.650442,0.650442,0.000000,NaN,0.000000,0.650442,0.650442,0.650442,0.650442
4,100007,NaN,0.322738,NaN,0,0.322738,0.322738,0.322738,0.322738,0.000000,NaN,0.000000,0.322738,0.322738,0.322738,0.322738


In [37]:
X_ext['EXT_SRC_weighted3'] = (X.EXT_SOURCE_1*2+X.EXT_SOURCE_2*3+X.EXT_SOURCE_3*4)/9
X_ext['EXT_SRC_weighted2'] = (X.EXT_SOURCE_1*3+X.EXT_SOURCE_2*4+X.EXT_SOURCE_3*2)/9
X_ext['EXT_SRC_weighted1'] = (X.EXT_SOURCE_1*4+X.EXT_SOURCE_2*2+X.EXT_SOURCE_3*3)/9

C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Umesh.k\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [38]:
X_ext.head()

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,external_sources_nanmin,external_sources_nanmax,external_sources_sum,external_sources_mean,external_sources_var,external_sources_median,external_sources_std,external_sources_nanmedian,external_sources_nanmean,external_sources_min,external_sources_max,EXT_SRC_weighted3,EXT_SRC_weighted2,EXT_SRC_weighted1
0,100002,0.083037,0.262949,0.139376,1,0.083037,0.262949,0.485361,0.161787,0.005646,0.139376,0.075139,0.139376,0.161787,0.083037,0.262949,0.168047,0.175517,0.141797
1,100003,0.311267,0.622246,NaN,0,0.311267,0.622246,0.933513,0.466757,0.024177,NaN,0.155489,0.466757,0.466757,0.311267,0.622246,NaN,NaN,NaN
2,100004,NaN,0.555912,0.729567,0,0.555912,0.729567,1.285479,0.642739,0.007539,NaN,0.086827,0.642739,0.642739,0.555912,0.729567,NaN,NaN,NaN
3,100006,NaN,0.650442,NaN,0,0.650442,0.650442,0.650442,0.650442,0.000000,NaN,0.000000,0.650442,0.650442,0.650442,0.650442,NaN,NaN,NaN
4,100007,NaN,0.322738,NaN,0,0.322738,0.322738,0.322738,0.322738,0.000000,NaN,0.000000,0.322738,0.322738,0.322738,0.322738,NaN,NaN,NaN


In [39]:
X_ext.to_csv("df_eda-external_sources",index=False)

In [40]:
# X_ext_corr = abs(X_ext.corr())

In [41]:
# X_ext_corr.sort_values('TARGET', ascending=False)['TARGET']

In [42]:
# sns.heatmap(X_ext_corr, 
#             xticklabels=X_ext_corr.columns,
# #             yticklabels=X_ext_corr.columns)

In [43]:
# X_train, X_test = train_test_split(X_ext)

In [44]:

# Y_train = X_train['TARGET']
# Y_test = X_test['TARGET']

# X_train = X_train.drop(columns='TARGET')
# X_test = X_test.drop(columns='TARGET')

In [45]:
# X_train = X_train.fillna(0)
# X_test = X_test.fillna(0)

In [46]:
# clf = tree.DecisionTreeClassifier()
# clf.fit(X_train, Y_train)

# print("R^2 on the train set:")
# print(clf.score(X_train, Y_train))

# print("\nR^2 on the test set:")
# print(clf.score(X_test, Y_test))

In [47]:
# X_train.head()

In [48]:

# feature_importances = pd.Series(clf.feature_importances_, index=X_train.columns.values)

In [49]:
# clf

In [50]:
# clf.feature_importances_

In [51]:
# feature_importances

In [52]:
# feature_importances.sort_values(ascending=False)